## Testing the ngram model

**We support:**
- n_grams of different length
- word vectors are weighted with tf-idf

**We need:**
- To be sure of the columns classes we want (Atm: `FIRTNAME, NAME, ADDRESS, CITY, DATE, CODE, ID, STRING`)
- A way to reduce the number of feature (maybe total number of occurence per feature)
- Scale to a column-like vision to enhance the distinction between STRING/FIRSTNAME/NAME/CODE to use redundancy
- In this vision, use the column name to confirm or enhance classification.


In [1]:
import ngram

In [23]:
clf = ngram.NGramClassifier()
X_train, y_train, X_test, y_test = clf.preprocess('data/columns')
clf.fit(X_train, y_train, ngram_range=(2, 3))
y_pred = clf.predict(X_test)
clf.score(y_pred, y_test)

(1000, 2453)
(200, 2453)
STRING              	16/17	   94.12% 	(FP:0)
NAME                	23/25	   92.0% 	(FP:35)
ID                  	34/34	   100.0% 	(FP:0)
ADDRESS             	24/24	   100.0% 	(FP:0)
CITY                	23/30	   76.67% 	(FP:3)
CODE                	12/25	   48.0% 	(FP:0)
DATE                	24/24	   100.0% 	(FP:0)
FIRSTNAME           	6/21	   28.57% 	(FP:0)
SCORE 162/200 :   81.0%
